# Part 4: Concurrency
Optimistic concurrency control (OCC) demo showing version conflict.


## 1. OCC conflict (two clients editing the same document)
Index -> read seq_no/primary_term -> update (A succeeds) -> update with stale version (B fails).


In [1]:
from elasticsearch import Elasticsearch, exceptions
from pprint import pprint

ES_HOST = 'http://localhost:9200'
INDEX = 'conc_demo'
DOC_ID = '1'
es = Elasticsearch(ES_HOST)
print('ES reachable?', es.ping())

# reset the index for a clean demo
if es.indices.exists(index=INDEX):
    es.indices.delete(index=INDEX)
es.indices.create(index=INDEX)
print('Created', INDEX)

# index initial doc
es.index(index=INDEX, id=DOC_ID, body={'counter': 1})
es.indices.refresh(index=INDEX)
print('Inserted doc with counter=1')


ES reachable? True
Created conc_demo
Inserted doc with counter=1


In [2]:
# retrieve seq_no + primary_term
doc = es.get(index=INDEX, id=DOC_ID)
seq_no = doc['_seq_no']
primary_term = doc['_primary_term']
print('Current seq_no:', seq_no, 'primary_term:', primary_term)

# simulate client A updating with the correct version
resp_a = es.index(
    index=INDEX,
    id=DOC_ID,
    body={'counter': 2},
    if_seq_no=seq_no,
    if_primary_term=primary_term
)
print('Client A update result:')
pprint(resp_a)

# simulate client B using the stale version -> expect conflict
try:
    es.index(
        index=INDEX,
        id=DOC_ID,
        body={'counter': 3},
        if_seq_no=seq_no,
        if_primary_term=primary_term
    )
    print('Client B unexpectedly succeeded')
except exceptions.ConflictError as e:
    print('Client B conflict (expected):', e.info['error']['type'])
    pprint(e.info['error'])


Current seq_no: 0 primary_term: 1
Client A update result:
ObjectApiResponse({'_index': 'conc_demo', '_id': '1', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 1, '_primary_term': 1})
Client B conflict (expected): version_conflict_engine_exception
{'index': 'conc_demo',
 'index_uuid': '9_mVUfikQCG1l7jJs6EwkA',
 'reason': '[1]: version conflict, required seqNo [0], primary term [1]. '
           'current document has seqNo [1] and primary term [1]',
 'root_cause': [{'index': 'conc_demo',
                 'index_uuid': '9_mVUfikQCG1l7jJs6EwkA',
                 'reason': '[1]: version conflict, required seqNo [0], primary '
                           'term [1]. current document has seqNo [1] and '
                           'primary term [1]',
                 'shard': '0',
                 'type': 'version_conflict_engine_exception'}],
 'shard': '0',
 'type': 'version_conflict_engine_exception'}


## 2. Lost update without OCC vs with OCC
Show read-modify-write races when not using `if_seq_no`/`if_primary_term`, then fix with OCC retries.


In [3]:
import threading, time

# reset index and seed counter=0
if es.indices.exists(index=INDEX):
    es.indices.delete(index=INDEX)
es.indices.create(index=INDEX)
es.index(index=INDEX, id=DOC_ID, body={'counter': 0})
es.indices.refresh(index=INDEX)
print('Reset index; counter=0')

def read_modify_write_no_occ(iterations):
    for _ in range(iterations):
        doc = es.get(index=INDEX, id=DOC_ID)
        current = doc['_source']['counter']
        es.index(index=INDEX, id=DOC_ID, body={'counter': current + 1})

threads = []
iters_per_thread = 50
num_threads = 5
for _ in range(num_threads):
    t = threading.Thread(target=read_modify_write_no_occ, args=(iters_per_thread,))
    threads.append(t)
    t.start()
for t in threads:
    t.join()

es.indices.refresh(index=INDEX)
final_doc = es.get(index=INDEX, id=DOC_ID)
expected = num_threads * iters_per_thread
print('Without OCC -> counter={}, expected={}'.format(final_doc['_source']['counter'], expected))


Reset index; counter=0
Without OCC -> counter=84, expected=250


In [4]:
# do the same with OCC retries to avoid lost updates
es.index(index=INDEX, id=DOC_ID, body={'counter': 0})
es.indices.refresh(index=INDEX)
print('Counter reset to 0 for OCC demo')

def occ_increment(iterations):
    for _ in range(iterations):
        while True:
            doc = es.get(index=INDEX, id=DOC_ID)
            seq_no = doc['_seq_no']
            primary_term = doc['_primary_term']
            current = doc['_source']['counter']
            try:
                es.index(
                    index=INDEX,
                    id=DOC_ID,
                    body={'counter': current + 1},
                    if_seq_no=seq_no,
                    if_primary_term=primary_term
                )
                break
            except exceptions.ConflictError:
                continue

threads = []
for _ in range(num_threads):
    t = threading.Thread(target=occ_increment, args=(iters_per_thread,))
    threads.append(t)
    t.start()
for t in threads:
    t.join()

es.indices.refresh(index=INDEX)
final_doc = es.get(index=INDEX, id=DOC_ID)
print('With OCC -> counter={}, expected={}'.format(final_doc['_source']['counter'], expected))


Counter reset to 0 for OCC demo
With OCC -> counter=250, expected=250


In [10]:
# increment counter atomically with server-side retry
es.index(index=INDEX, id=DOC_ID, body={'counter': 0})
es.indices.refresh(index=INDEX)
print('Counter reset to 0 for OCC demo')
threads = []

def occ_increment(iters):
    for _ in range(iters):
        es.update(
            index=INDEX,
            id=DOC_ID,
            body={"script": "ctx._source.counter += 1"},
            retry_on_conflict=10  # number of auto-retries if a conflict occurs
        )

for _ in range(num_threads):
    t = threading.Thread(target=occ_increment, args=(iters_per_thread,))
    threads.append(t)
    t.start()
for t in threads:
    t.join()

es.indices.refresh(index=INDEX)
final_doc = es.get(index=INDEX, id=DOC_ID)
print('With OCC -> counter={}, expected={}'.format(final_doc['_source']['counter'], expected))

Counter reset to 0 for OCC demo
With OCC -> counter=250, expected=250
